In [1]:
# %pip install python-mecab-ko

Note: you may need to restart the kernel to use updated packages.


In [42]:
sentences = [
    "오늘 날씨가 좋아서 나들이 가고 싶다.", # -> [오늘, 날씨가, 좋아서 ...]
    "이 영화는 정말 재미있었어요.",
    "맛있는 음식을 먹으러 갈까요?",
    "운동을 하면 건강에 좋아지는 것 같아요.",
    "공부하기 싫어서 미루고 있어요.",
    "여행 계획을 세우고 있는데 어디로 갈까요?",
    "좋은 책을 읽으면 마음이 편안해져요.",
    "오늘은 친구들과 만나서 재미있게 놀았어요.",
    "새로운 언어를 배우는 것은 어려워도 흥미로워요.",
    "주말에 가족들과 함께 시간을 보내기로 했습니다."
]

### 자연어 전처리

In [43]:
from mecab import MeCab
mecab = MeCab()
# 형태분석기 사용해 명사만 추출
morphs_list = list()
for sentence in sentences :
    morphs = mecab.nouns(sentence)
    morphs_list = morphs_list + morphs
morphs_list

['날씨',
 '나',
 '영화',
 '음식',
 '운동',
 '건강',
 '것',
 '공부',
 '여행',
 '계획',
 '어디',
 '책',
 '마음',
 '편안',
 '오늘',
 '친구',
 '언어',
 '것',
 '흥미',
 '주말',
 '가족',
 '시간']

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=True)
features = vectorizer.fit_transform(morphs_list)

### LDA 적용

In [45]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=3, random_state=111)

In [46]:
lda_model.fit(features)

LatentDirichletAllocation(n_components=3, random_state=111)

In [47]:
lda_model.components_

array([[0.33654237, 0.33714206, 0.33654245, 1.32934725, 0.33714184,
        0.33654233, 1.32934644, 0.33714199, 0.33654233, 0.33714199,
        0.33714194, 1.32934726, 0.33714188, 1.32934726, 0.33654235,
        0.33714198, 0.33714192, 0.33654237],
       [0.33556197, 1.32641014, 0.33556199, 0.33516318, 1.32641046,
        0.33556197, 0.33516348, 1.32641023, 0.33556196, 1.32641026,
        1.32641033, 0.33516318, 1.32641041, 0.33516318, 0.33556197,
        1.32641028, 1.32641037, 0.33556197],
       [1.32789566, 0.33644779, 1.32789556, 0.33548956, 0.3364477 ,
        1.3278957 , 0.33549008, 0.33644778, 1.32789571, 0.33644775,
        0.33644774, 0.33548956, 0.33644771, 0.33548956, 1.32789568,
        0.33644774, 0.33644771, 1.32789566]])

In [48]:
# lda_model.components_
# len(lda_model.components_[0])
topics_list = lda_model.components_[0]  # 첫번째 토픽
topics_list

array([0.33654237, 0.33714206, 0.33654245, 1.32934725, 0.33714184,
       0.33654233, 1.32934644, 0.33714199, 0.33654233, 0.33714199,
       0.33714194, 1.32934726, 0.33714188, 1.32934726, 0.33654235,
       0.33714198, 0.33714192, 0.33654237])

### 토픽별 단어 표시

In [49]:
dictionary_list = vectorizer.get_feature_names_out()
dictionary_list 
# len(vectorizer.get_feature_names_out())

array(['가족', '건강', '계획', '공부', '날씨', '마음', '시간', '어디', '언어', '여행', '영화',
       '오늘', '운동', '음식', '주말', '친구', '편안', '흥미'], dtype=object)

In [50]:
import pandas as pd
df_datas = [topics_list, dictionary_list]
df_topics = pd.DataFrame(data=df_datas)
df_topics = df_topics.T
df_topics[:2]

,0,1
0,0.336542,가족
1,0.337142,건강


In [51]:
df_topics.columns

RangeIndex(start=0, stop=2, step=1)

In [52]:
df_topics.sort_values(0, ascending=False)

,0,1
13,1.329347,음식
11,1.329347,오늘
3,1.329347,공부
6,1.329346,시간
1,0.337142,건강
7,0.337142,어디
9,0.337142,여행
15,0.337142,친구
10,0.337142,영화
16,0.337142,편안


### 댓글과 주요 토픽 연결

In [77]:
## 상위 단어 추출
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics = df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print (df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:4])# get values from Serise
    topics_text
    print (topics_text)

음식 오늘 공부 시간
날씨 운동 편안 영화
언어 마음 주말 흥미


In [72]:
df_topics[1].values[:4]

array(['언어', '마음', '주말', '흥미'], dtype=object)

In [74]:
' '.join(df_topics[1].values[:4])

'언어 마음 주말 흥미'

In [54]:
topics_output = lda_model.transform(features)
topics_output

array([[0.16857425, 0.66320011, 0.16822564],
       [0.33333333, 0.33333333, 0.33333333],
       [0.16857425, 0.66320011, 0.16822564],
       [0.66467306, 0.1675818 , 0.16774513],
       [0.16857425, 0.66320011, 0.16822564],
       [0.16857425, 0.6632001 , 0.16822565],
       [0.33333333, 0.33333333, 0.33333333],
       [0.66467306, 0.1675818 , 0.16774513],
       [0.16857425, 0.66320011, 0.16822565],
       [0.16827277, 0.16778152, 0.66394571],
       [0.16857425, 0.66320011, 0.16822565],
       [0.33333333, 0.33333333, 0.33333333],
       [0.16827277, 0.16778152, 0.66394571],
       [0.16857425, 0.66320011, 0.16822564],
       [0.66467306, 0.1675818 , 0.16774513],
       [0.16857425, 0.66320011, 0.16822564],
       [0.16827277, 0.16778152, 0.66394571],
       [0.33333333, 0.33333333, 0.33333333],
       [0.16827277, 0.16778152, 0.66394571],
       [0.16827277, 0.16778152, 0.66394571],
       [0.16827277, 0.16778152, 0.66394571],
       [0.66467305, 0.16758181, 0.16774514]])